In [1]:
import os
import pandas as pd

import json

from collections import defaultdict

In [2]:
data_filename = "/home/zengs/data/Code/reproduce/protgps/data/dataset.json"

data_json = json.load(open(data_filename))


In [3]:
LABEL_MAP =  {
  "transcriptional" :  "TRANSCRIPTIONAL",
  "chromosomes" :  "CHROMOSOME",
  "chromosome" :  "CHROMOSOME",
  "nuclear_pore_complex" :  "NUCLEAR_PORE_COMPLEX",
  "nuclear_speckles" :  "NUCLEAR_SPECKLE",
  "nuclear_speckle" :  "NUCLEAR_SPECKLE",
  "pbody" :  "P-BODY",
  "p-body" :  "P-BODY",
  "pml_body" :  "PML-BDOY",
  "pml-bdoy" :  "PML-BDOY",
  "post_synaptic_density" :  "POST_SYNAPTIC_DENSITY",
  "stress_granule" :  "STRESS_GRANULE",
  "nucleolus" :  "NUCLEOLUS",
  "cajal_body" :  "CAJAL_BODY",
  "rna_granule" :  "RNA_GRANULE",
  "cell_junction" :  "CELL_JUNCTION"
}

LABEL_COLS = ['TRANSCRIPTIONAL',
 'CHROMOSOME',
 'NUCLEAR_PORE_COMPLEX',
 'NUCLEAR_SPECKLE',
 'P-BODY',
 'PML-BDOY',
 'POST_SYNAPTIC_DENSITY',
 'STRESS_GRANULE',
 'NUCLEOLUS',
 'CAJAL_BODY',
 'RNA_GRANULE',
 'CELL_JUNCTION']

In [4]:
num_entry = 0
num_entry_name = 0

temp_data = None

data_res = defaultdict(list)
for one_data in data_json:
  if "entry" in one_data:
    data_res["entry"].append(one_data["entry"])
    data_res["split"].append(one_data["split"])
    data_res["sequence"].append(one_data["sequence"])
    label_dict = one_data["labels"]
    
    updated_label_dict = {}
    for key, value in label_dict.items():
      if key in LABEL_MAP:
        if LABEL_MAP[key] == "CHROMOSOME" or LABEL_MAP[key] == "NUCLEAR_SPECKLE":
          continue
        updated_label_dict[LABEL_MAP[key]] = int(value)
    
    updated_label_dict["CHROMOSOME"] = int((one_data["labels"]["chromosomes"] + one_data["labels"]["chromosome"]) > 0)
    updated_label_dict["NUCLEAR_SPECKLE"] = int(one_data["labels"]["nuclear_speckle"] > 0)
    data_res["labels"].append(updated_label_dict)
    
    # if one_data["entry"] == "O00422":
    #   print(one_data)
    #   break
    
  elif "Entry Name" in one_data:
    data_res["entry"].append(one_data["Entry"])
    data_res["split"].append(one_data["split"])
    data_res["sequence"].append(one_data["Sequence"])
    label_dict = one_data["labels"]
    
    updated_label_dict = {}
    for key, value in label_dict.items():
      if key in LABEL_MAP:
        updated_label_dict[LABEL_MAP[key]] = int(value)

    data_res["labels"].append(updated_label_dict)
    
    # if one_data["Entry"] == "O00422":
    #   print(one_data)
    #   print(updated_label_dict)
    #   break

data_res_df = pd.DataFrame(data_res)

In [5]:
data_res_df[data_res_df.entry == "BUB1B_HUMAN"]["labels"].values[0]

{'CAJAL_BODY': 0,
 'NUCLEOLUS': 0,
 'P-BODY': 0,
 'PML-BDOY': 0,
 'STRESS_GRANULE': 0,
 'CELL_JUNCTION': 0,
 'NUCLEAR_PORE_COMPLEX': 0,
 'RNA_GRANULE': 0,
 'CHROMOSOME': 1,
 'NUCLEAR_SPECKLE': 0}

In [6]:

df_expanded = pd.json_normalize(data_res_df['labels'])
df_expanded = df_expanded[LABEL_COLS]
df_final = data_res_df.drop(columns=['labels']).join(df_expanded)
df_final.fillna(0, inplace=True)


In [7]:
for col in LABEL_COLS:
  df_final[col] = df_final[col].astype(int)
  
df_final = df_final[[col for col in df_final.columns if col != 'sequence'] + ['sequence']]
df_final.to_csv("dataset_from_json.csv", index=False)
df_final.to_excel("dataset_from_json.xlsx", index=False)